In [ ]:
import numpy as np
import pandas as pd

def linear_kernel(x1, x2):
  return np.dot(x1, x2)

def polynomial_kernel(x1, x2, p):
  return (np.dot(x1, x2))**p

def rbf_kernel(x1, x2, sigma):
  return np.exp(-np.linalg.norm(x1-x2)**2 / (2 * (sigma ** 2)))

def holdout(data, trainp, validp):
  m = np.shape(data)[0]

  train = data[0 : int(np.floor(m*trainp/100))]
  valid = data[int(np.floor(m*trainp/100)) : int(np.floor(m*trainp/100))+int(np.floor(m*validp/100))]
  test = data[int(np.floor(m*trainp/100))+int(np.floor(m*validp/100)) : None]

  y_train = train[:,-1] #shape = (rows,)
  x_train = np.delete(train, -1, axis=1)

  y_valid = valid[:,-1] #shape = (rows,)
  x_valid = np.delete(valid, -1, axis=1)

  y_test = test[:,-1] #shape = (rows,)
  x_test = np.delete(test, -1, axis=1)

  return x_train,x_valid,x_test,y_train,y_valid,y_test

def appendones(x):
  m = np.shape(x)[0]
  return np.concatenate((np.ones((m,1)),x), axis=1)

def conf_matrix(y,pred):
  m = np.zeros(4)
  for i in range(len(y)):
    if y[i]==-1 and pred[i]==-1:
      m[0] += 1
    if y[i]==-1 and pred[i]==1:
      m[1] += 1
    if y[i]==1 and pred[i]==-1:
      m[2] += 1
    if y[i]==1 and pred[i]==1:
      m[3] += 1
  return m

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/NNFL Assignments (Aug 2021)/Assignment 2/data55.xlsx')
cols = np.array(data.columns, ndmin=2)
data = data.to_numpy()
data = np.concatenate((cols,data), axis=0)

# shuffle & test-valid-split
np.random.seed(0)
np.random.shuffle(data)
x_train, x_valid, x_test, y_train, y_valid, y_test = holdout(data, 70, 10)

# # normalizing
# mu = np.mean(x_train, axis=0)
# std = np.std(x_train, axis=0)

# x_train = (x_train-mu)/std
# x_valid = (x_valid-mu)/std
# x_test = (x_test-mu)/std

# appending ones (input for bias)
x_train = appendones(x_train)
x_valid = appendones(x_valid)
x_test = appendones(x_test)

n = np.shape(x_train)[1]
m_train = np.size(y_train)
m_valid = np.size(y_valid)
m_test = np.size(y_test)

# changing classes from {0,1} to {-1,1}
y_train = np.where(y_train==0, -1, 1)
y_valid = np.where(y_valid==0, -1, 1)
y_test = np.where(y_test==0, -1, 1)

In [ ]:
 # using linear kernel
# gram matrix 
K = np.zeros((m_train,m_train))
for i in range(m_train):
  for j in range(m_train):
    K[i,j] = linear_kernel(x_train[i], x_train[j])

# training
delta = np.zeros(m_train)
h = np.zeros(m_train) # hypothesis
for i in range(m_train):
  h[i] = np.sign(np.sum(delta * y_train * K[:,i]))
  if h[i] != y_train[i]:
    delta[i] += 1

# testing
pred = np.zeros(m_test)
for i in range(m_test):
  pred[i] = np.sign(np.sum(delta * y_train * linear_kernel(np.array(x_test[i], ndmin=2),x_train.T)))

# performance measures
tn,fp,fn,tp = conf_matrix(y_test, pred)

sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
accuracy = (tp+tn)/(tp+tn+fp+fn)
print("Accuracy = {}".format(accuracy))
print("Sensitivity = {}".format(sensitivity))
print("Specificity = {}".format(specificity))


Accuracy = 0.627906976744186
Sensitivity = 1.0
Specificity = 0.15789473684210525


In [ ]:
# using polynomial kernel (degree = 2)
# gram matrix 
K = np.zeros((m_train,m_train))
for i in range(m_train):
  for j in range(m_train):
    K[i,j] = polynomial_kernel(x_train[i], x_train[j], 2)

# training
delta = np.zeros(m_train)
h = np.zeros(m_train) # hypothesis
for i in range(m_train):
  h[i] = np.sign(np.sum(delta * y_train * K[:,i]))
  if h[i] != y_train[i]:
    delta[i] += 1

# testing
pred = np.zeros(m_test)
for i in range(m_test):
  pred[i] = np.sign(np.sum(delta * y_train * polynomial_kernel(np.array(x_test[i], ndmin=2),x_train.T, 2)))

# performance measures
tn,fp,fn,tp = conf_matrix(y_test, pred)

sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
accuracy = (tp+tn)/(tp+tn+fp+fn)
print("Accuracy = {}".format(accuracy))
print("Sensitivity = {}".format(sensitivity))
print("Specificity = {}".format(specificity))

Accuracy = 0.813953488372093
Sensitivity = 0.7083333333333334
Specificity = 0.9473684210526315


In [ ]:
# using RBF kernel (standard deviation = 2.55)
# gram matrix 
K = np.zeros((m_train,m_train))
for i in range(m_train):
  for j in range(m_train):
    K[i,j] = rbf_kernel(x_train[i], x_train[j], 2.55)

# training
delta = np.zeros(m_train)
h = np.zeros(m_train) # hypothesis
for i in range(m_train):
  h[i] = np.sign(np.sum(delta * y_train * K[:,i]))
  if h[i] != y_train[i]:
    delta[i] += 1

# testing
pred = np.zeros(m_test)
for i in range(m_test):
  pred[i] = np.sign(np.sum(delta * y_train * rbf_kernel(np.array(x_test[i], ndmin=2),x_train, 2.55)))

# performance measures
tn,fp,fn,tp = conf_matrix(y_test, pred)

sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
accuracy = (tp+tn)/(tp+tn+fp+fn)
print("Accuracy = {}".format(accuracy))
print("Sensitivity = {}".format(sensitivity))
print("Specificity = {}".format(specificity))

Accuracy = 0.8333333333333334
Sensitivity = 0.6666666666666666
Specificity = 1.0
